In [1]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local").appName("demo").getOrCreate()

24/09/19 16:30:34 WARN Utils: Your hostname, Matthews-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.15.118 instead (on interface en0)
24/09/19 16:30:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/19 16:30:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
transactions = spark.createDataFrame([
    (1, "socks", 1), 
    (2, "chips", 1),
    (3, "air conditioner", None),
    (4, "tea", None),
], ["transaction_id", "item_name", "user_id"])

In [14]:
transactions.show()

+--------------+---------------+-------+
|transaction_id|      item_name|user_id|
+--------------+---------------+-------+
|             1|          socks|      1|
|             2|          chips|      1|
|             3|air conditioner|   NULL|
|             4|            tea|   NULL|
+--------------+---------------+-------+



In [8]:
users = spark.createDataFrame([
    (1, "Smith"), 
    (2, "Jones"),
    (None, "UNKNOWN"),
], ["user_id", "last_name"])

In [15]:
users.show()

+-------+---------+
|user_id|last_name|
+-------+---------+
|      1|    Smith|
|      2|    Jones|
|   NULL|  UNKNOWN|
+-------+---------+



In [10]:
transactions.join(
    users, 
    transactions.user_id == users.user_id,
).show()

+--------------+---------+-------+-------+---------+
|transaction_id|item_name|user_id|user_id|last_name|
+--------------+---------+-------+-------+---------+
|             1|    socks|      1|      1|    Smith|
|             2|    chips|      1|      1|    Smith|
+--------------+---------+-------+-------+---------+



In [13]:
transactions.join(
    users, 
    transactions.user_id.eqNullSafe(users.user_id),
).show()

+--------------+---------------+-------+-------+---------+
|transaction_id|      item_name|user_id|user_id|last_name|
+--------------+---------------+-------+-------+---------+
|             3|air conditioner|   NULL|   NULL|  UNKNOWN|
|             4|            tea|   NULL|   NULL|  UNKNOWN|
|             1|          socks|      1|      1|    Smith|
|             2|          chips|      1|      1|    Smith|
+--------------+---------------+-------+-------+---------+



In [16]:
df = spark.createDataFrame([
    (1, 1), 
    (2, 99),
    (None, None),
], ["col_a", "col_b"])

In [18]:
df.withColumn("==", df.col_a==df.col_b).withColumn("eqNullSafe", df.col_a.eqNullSafe(df.col_b)).show()

+-----+-----+-----+----------+
|col_a|col_b|   ==|eqNullSafe|
+-----+-----+-----+----------+
|    1|    1| true|      true|
|    2|   99|false|     false|
| NULL| NULL| NULL|      true|
+-----+-----+-----+----------+

